In [1]:
import pandas as pd
import numpy as np
import glob

# US ITC Dynamic Gravity dataset
- <b>Goal</b>: Compile the dynamic gravity dataset to show inter-country trade relationships. This result will be further normalized with the importer or exporter share of resources in the BACI trade dataset, which will then be used for regression analysis. 
- This dataset measures the trade friction between countries. It includes 286 countries from 1948-2019. Intra-country trade is removed to reduce the data size.
- The analysis here is inspired by Shepard et al 2022: https://github.com/junukitashepard/message_trade/blob/master/analysis/2_regressions/2_add_gravity_terms.R
    1. agree cu: Country pair is in at least one customs union 
    2. agree eia: Country pair is in at least one economic integration agreement 
    3. agree fta: Country pair is in at least one free trade agreement 
    4. agree psa: Country pair is in at least one partial scope agreement 
    5. agree pta: Country pair is in at least one active preferential trade agreement
    6. contiguity: Country pair shares a common border
    7. common language: Residents of country pair speak at least one common language
    8. colony of destination: ever Origin country was ever a colony of the destination country 
    9. colony of origin: ever Destination country was ever a colony of the origin country
    10. sanction threat: There exists a threat of sanction between one country in a record towards the other
    11. sanction imposition: There exists a sanction between one country in a record towards the other
- <b>Assumption</b>:
    - Use 2019 data for 2020 and 2021 --> so we can use the BACI dataset that carries for longer

In [2]:
DGD_dir = "../../data/global_USITC_DGD/"

In [3]:
dirpath = DGD_dir+"release_2.0/*.csv"
fpaths = glob.glob(dirpath)
fpaths

['../../data/global_USITC_DGD/release_2.0\\release_2.0_1948_1959.csv',
 '../../data/global_USITC_DGD/release_2.0\\release_2.0_1960_1969.csv',
 '../../data/global_USITC_DGD/release_2.0\\release_2.0_1970_1979.csv',
 '../../data/global_USITC_DGD/release_2.0\\release_2.0_1980_1989.csv',
 '../../data/global_USITC_DGD/release_2.0\\release_2.0_1990_1999.csv',
 '../../data/global_USITC_DGD/release_2.0\\release_2.0_2000_2004.csv',
 '../../data/global_USITC_DGD/release_2.0\\release_2.0_2005_2009.csv',
 '../../data/global_USITC_DGD/release_2.0\\release_2.0_2010_2014.csv',
 '../../data/global_USITC_DGD/release_2.0\\release_2.0_2015_2019.csv']

In [4]:
# variables used by Shepard et al 2022
var_list = ['year', 'iso3_o', 'iso3_d', 
            'contiguity', 'agree_cu', 'agree_eia', 'agree_fta', 'agree_psa', 'agree_pta', 
            'sanction_threat', 'sanction_imposition',
            'common_language', 'colony_of_destination_ever', 'colony_of_origin_ever']

In [5]:
for file in fpaths:
    file_data = pd.read_csv(file)
    file_data_select = file_data[var_list]
    file_data_select = file_data_select[file_data_select["iso3_o"]!=file_data_select["iso3_d"]]
    
    if '2019' in file:
        # assume 2020 and 2021 data the same as 2019 data
        file_data_2019 = file_data_select[file_data_select["year"]==2019]
        for new_year in [2020,2021]:
            file_data_new = file_data_2019.copy()
            file_data_new["year"] = new_year
            file_data_select = pd.concat([file_data_select,file_data_new])

    file_data_select = file_data_select.set_index("year")
    file_data_select.to_csv("_temporal_dynamic_gravity"+file[-14:])
file_data_select

,iso3_o,iso3_d,contiguity,agree_cu,agree_eia,agree_fta,agree_psa,agree_pta,sanction_threat,sanction_imposition,common_language,colony_of_destination_ever,colony_of_origin_ever
year,,,,,,,,,,,,,
2015,ABW,AFG,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,0,0.0,0.0
2016,ABW,AFG,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,0,0.0,0.0
2017,ABW,AFG,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,0,0.0,0.0
2018,ABW,AFG,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,0,0.0,0.0
2019,ABW,AFG,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021,ZWE,WLF,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,0,0.0,0.0
2021,ZWE,WSM,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,1,0.0,0.0
2021,ZWE,YEM,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,0,0.0,0.0
